# Homework Assignment 2: Data Prep
In this homework assignment, you will continue your exploration of the [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM), described in the paper found [here](https://doi.org/10.1038/s41597-020-0548-x).


This assignment will have you explore the cardinalities, number of missing values, detect outliers, handle missing values and outliers, and create data quality report for original and cleaned dataset.

## Step 1: Downloading the Data

This assignment will only be using [Partition 1](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/BMXYCB). Recall that in Homework 1, we started to construct the analytics base table for our [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM). In that assignment, we read the data from the two subdirectories, __FL__ and __NF__, of the __partition1__ direcotry. These two subdirectories represented the two classes of our target feature in the solar flare prediction problem we will be attempting to solve this semester. We then processed these samples of multivariate time series to construct descriptive features for each sample, and then placed them into our analytics base table.

In this assignment, you will be utilizing a set of extracted descriptive features much like what you were asked to construct in Homework 1. However, this dataset contains many more extracted features than you were asked to compute for Homework 1 (>800), so we need to explore the data to find data quality issues and identify ways to address these issues. Below are links to the full extracted feature dataset for partition 1 and a toy dataset to use for testing you functions.

- [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv)
- [Toy Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_partition1ExtractedFeatures.csv)

Now that you have the extracted features csv files, you will load that data into a Pandas DataFrame using the [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) method.  


In [4]:
import os
import glob
import pandas as pd
from pandas import DataFrame 
import numpy as np

### Read the dataset

In [5]:
data_dir = r'C:/Users/akash/OneDrive/Desktop/Data Science/hw2'
data_file = r'toy_partition1ExtractedFeatures.csv'

In [6]:
abt = pd.read_csv(os.path.join(data_dir, data_file), index_col=0)

### Q1 (25 points)

Write a function to extract the various pieces of a data quality report, for a specific attribute, and return a dataframe with this information.

 * 'Feature Name': Contains the time series statistical feature name
 
 * 'Cardinality': Contains the count of unique values for the feature
            
 * 'Non-null Count': Contains the number of non-null entries for the feature
            
 * 'Null Count': Contains the number of null or missing entries for the feature
            
 * 'Min': Contains the minimum value of the feature (Without considering the null or nan value)
 
 * '25th': Contains the first quartile (25%) of the feature values (Without considering the null/nan value)
 
 * 'Mean': Contains the mean of the feature values (Without considering the null/nan value)
 
 * '50th': Contains the median of the feature values (Without considering the null/nan value)
            
 * '75th': Contains the third quartile (75%) of the feature values (Without considering the null/nan value)
 
 * 'Max': Contains the maximum value of the feature (Without considering the null/nan value),
            
 * 'Std. Dev': Contains the standard deviation of the feature (Without considering the null/nan value)
 
In addition to the values above, you should identify the number of upper and lower outliers using the $val < Q1-1.5IQR$ and $val > Q3+1.5IQR$ outlier identification method. These added features should be called `Outlier Count Low` and `Outliers Count High` respectively.


 
 Some useful functions for this can be found at:
 
 * [Numpy.percentile](https://numpy.org/doc/stable/reference/generated/numpy.percentile.html)
 
 * [pandas.isna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isna.html)
 
 * [Numpy.mean](https://numpy.org/doc/stable/reference/generated/numpy.mean.html)
 
 * [Numpy.std](https://numpy.org/doc/stable/reference/generated/numpy.std.html)
 

In [2]:
def calc_summary_for(feature_name:str, data:DataFrame) -> DataFrame:
    summary_feature_names = ['Feature Name', 'Cardinality', 'Non-null Count', 'Null Count', 'Min', '25th', 'Mean', 
                             '50th', '75th', 'Max', 'Outlier Count Low', 'Outlier Count High' 'Std. Dev']
    IQR = (data[feature_name].quantile(q=.75)) - (data[feature_name].quantile(q=.25))*1.5 
    o_low = 0
    o_high = 0
    for i in data[feature_name].values:
        if i < (data[feature_name].quantile(q=.25) - IQR):
            o_low+= 1
        elif i > (data[feature_name].quantile(q=.75) + IQR):
            o_high+= 1
        else:
            continue

    
    
    df = pd.DataFrame()
    df['Feature Name'] = [feature_name]
    df['Cardinality'] = [data[feature_name].nunique()]
    df['Non-null Count'] = [data[feature_name].notnull().sum()]
    df['Null Count'] = [data[feature_name].isnull().sum()]
    df['Min'] = [data[feature_name].min()]
    df['25th'] = [data[feature_name].quantile(q=.25)]
    df['Mean'] = [data[feature_name].mean()]
    df['50th'] = [data[feature_name].quantile(q=.50)]
    df['75th'] = [data[feature_name].quantile(q=.75)]
    df['Max'] = [data[feature_name].max()]
    df['Outlier Count Low'] = [o_low]
    df['Outlier Count High'] = [o_high]
    df['Std.Dev'] = [data[feature_name].std()]
    df.set_index(['Feature Name'], inplace=True)
    
    print (df)
    return df

    
    
    # Your answer to Q1 goes here! Make sure to return a DataFrame with the features specified. 

NameError: name 'DataFrame' is not defined

In [3]:
import pandas as pd
abt = pd.read_csv(os.path.join(data_dir, data_file), index_col=0)
#results = calc_summary_for(feature_name, data)
print(abt)
summary_TOTUSJH_min = calc_summary_for('TOTUSJH_min', abt)

NameError: name 'data_dir' is not defined

### Q2 (25 points)
Using what you produced to answer Q1, you should now write a function to construct the data quality report for all of the numerical features of our dataset.  You should loop over all of the features in the analytics base table represented by the input feature dataset files from partition 1, with the exception of the first column (this is the index column if you read the file correctly), and the `id`, `lab`, `st`, and `et` columns.  

Your output from this function will be a DataFrame that has 1 row for each feature. 

In [14]:
import pandas as pd
from pandas import DataFrame 
def construct_quality_report(data:DataFrame) -> DataFrame: 
    excluded_columns = ['id', 'lab', 'st', 'et']
    
    summary_feature_names = ['Feature Name', 'Cardinality', 'Non-null Count', 'Null Count', 'Min', '25th', 'Mean', 
                             '50th', '75th', 'Max', 'Outlier Count Low', 'Outlier Count High']
    data.drop(columns = excluded_columns, inplace = True)
    the_list = []
    for feature_name,values in data.iteritems():
        the_list.append(calc_summary_for(feature_name,data))
        
    return pd.concat(the_list)
    
    # Your answer to Q2 goes here! Make sure to return a DataFrame with the features specified.

In [15]:
summary_table = construct_quality_report(abt)

              Cardinality  Non-null Count  Null Count        Min        25th  \
Feature Name                                                                   
TOTUSJH_min            50              50           0  13.292757  188.084597   

                     Mean         50th         75th          Max  \
Feature Name                                                       
TOTUSJH_min   1474.046501  1302.193667  2307.005998  4807.254827   

              Outlier Count Low      Std.Dev  
Feature Name                                  
TOTUSJH_min                   3  1396.206806  
              Cardinality  Non-null Count  Null Count        Min        25th  \
Feature Name                                                                   
TOTUSJH_max            48              50           0  29.150427  254.628298   

                     Mean         50th         75th          Max  \
Feature Name                                                       
TOTUSJH_max   1762.417043  1629.3804

                            Cardinality  Non-null Count  Null Count      Min  \
Feature Name                                                                   
TOTUSJH_difference_of_stds           50              50           0  0.11354   

                                25th       Mean      50th       75th  \
Feature Name                                                           
TOTUSJH_difference_of_stds  2.693618  27.410339  15.13461  38.353624   

                                  Max  Outlier Count Low    Std.Dev  
Feature Name                                                         
TOTUSJH_difference_of_stds  162.75267                  6  34.660943  
                            Cardinality  Non-null Count  Null Count       Min  \
Feature Name                                                                    
TOTUSJH_difference_of_vars           50              50           0  1.920643   

                                 25th         Mean         50th         75th  \
Feature 

                                 Cardinality  Non-null Count  Null Count  \
Feature Name                                                               
TOTUSJH_average_absolute_change           50              50           0   

                                      Min      25th       Mean       50th  \
Feature Name                                                                
TOTUSJH_average_absolute_change  1.363014  5.858084  18.746018  15.262293   

                                      75th         Max  Outlier Count Low  \
Feature Name                                                                
TOTUSJH_average_absolute_change  23.324948  133.718855                  3   

                                   Std.Dev  
Feature Name                                
TOTUSJH_average_absolute_change  24.042701  
                                            Cardinality  Non-null Count  \
Feature Name                                                              
TOTUSJH_average_absolu

              Cardinality  Non-null Count  Null Count           Min  \
Feature Name                                                          
TOTBSQ_var             50              50           0  2.068391e+14   

                      25th          Mean          50th          75th  \
Feature Name                                                           
TOTBSQ_var    3.790878e+16  1.780098e+18  2.370520e+17  1.268338e+18   

                       Max  Outlier Count Low       Std.Dev  
Feature Name                                                 
TOTBSQ_var    2.138121e+19                 10  4.056376e+18  
                 Cardinality  Non-null Count  Null Count       Min      25th  \
Feature Name                                                                   
TOTBSQ_skewness           50              50           0 -1.576371 -0.393113   

                     Mean      50th     75th       Max  Outlier Count Low  \
Feature Name                                                     

                         Cardinality  Non-null Count  Null Count  \
Feature Name                                                       
TOTBSQ_dderivative_mean           50              50           0   

                                  Min          25th          Mean  \
Feature Name                                                        
TOTBSQ_dderivative_mean -5.428790e+07 -2.474726e+06  2.438978e+07   

                                 50th          75th           Max  \
Feature Name                                                        
TOTBSQ_dderivative_mean  2.104384e+06  2.877844e+07  2.239146e+08   

                         Outlier Count Low       Std.Dev  
Feature Name                                              
TOTBSQ_dderivative_mean                 10  6.051613e+07  
                         Cardinality  Non-null Count  Null Count  \
Feature Name                                                       
TOTBSQ_gderivative_mean           50              50           0  

...

               Cardinality  Non-null Count  Null Count      Min     25th  \
Feature Name                                                               
EPSX_skewness           50              50           0 -1.46143 -0.15346   

                   Mean      50th      75th       Max  Outlier Count Low  \
Feature Name                                                               
EPSX_skewness  0.112465  0.112511  0.335116  1.046957                  3   

                Std.Dev  
Feature Name             
EPSX_skewness  0.464043  
               Cardinality  Non-null Count  Null Count       Min      25th  \
Feature Name                                                                 
EPSX_kurtosis           50              50           0 -1.501509 -1.175366   

                   Mean      50th      75th       Max  Outlier Count Low  \
Feature Name                                                               
EPSX_kurtosis -0.808073 -0.915536 -0.636932  1.739601                  2   



                           Cardinality  Non-null Count  Null Count       Min  \
Feature Name                                                                   
EPSX_dderivative_skewness           50              50           0 -1.129076   

                               25th      Mean     50th      75th       Max  \
Feature Name                                                                 
EPSX_dderivative_skewness -0.111385  0.010812  0.04115  0.211767  0.622376   

                           Outlier Count Low   Std.Dev  
Feature Name                                            
EPSX_dderivative_skewness                  1  0.333726  
                           Cardinality  Non-null Count  Null Count       Min  \
Feature Name                                                                   
EPSX_gderivative_skewness           50              50           0 -0.651365   

                               25th      Mean      50th      75th       Max  \
Feature Name                     

              Cardinality  Non-null Count  Null Count  Min  25th      Mean  \
Feature Name                                                                 
R_VALUE_min            35              50           0  0.0   0.0  2.996308   

                  50th      75th       Max  Outlier Count Low   Std.Dev  
Feature Name                                                             
R_VALUE_min   4.256409  4.700478  5.519059                  0  2.152525  
              Cardinality  Non-null Count  Null Count  Min      25th  \
Feature Name                                                           
R_VALUE_max            46              50           0  0.0  3.143049   

                  Mean      50th      75th       Max  Outlier Count Low  \
Feature Name                                                              
R_VALUE_max   3.857418  4.500024  4.872757  5.628571                  7   

               Std.Dev  
Feature Name            
R_VALUE_max   1.577087  
                Cardinali

                            Cardinality  Non-null Count  Null Count  Min  \
Feature Name                                                               
R_VALUE_difference_of_stds           46              50           0  0.0   

                                25th      Mean      50th      75th       Max  \
Feature Name                                                                   
R_VALUE_difference_of_stds  0.008352  0.109151  0.017795  0.074421  1.090285   

                            Outlier Count Low   Std.Dev  
Feature Name                                             
R_VALUE_difference_of_stds                 10  0.213645  
                            Cardinality  Non-null Count  Null Count  Min  \
Feature Name                                                               
R_VALUE_difference_of_vars           46              50           0  0.0   

                                25th      Mean      50th      75th       Max  \
Feature Name                                   

                                 Cardinality  Non-null Count  Null Count  Min  \
Feature Name                                                                    
R_VALUE_average_absolute_change           46              50           0  0.0   

                                     25th    Mean      50th      75th  \
Feature Name                                                            
R_VALUE_average_absolute_change  0.013259  0.0758  0.020158  0.067188   

                                      Max  Outlier Count Low   Std.Dev  
Feature Name                                                            
R_VALUE_average_absolute_change  0.537999                 10  0.119396  
                                            Cardinality  Non-null Count  \
Feature Name                                                              
R_VALUE_average_absolute_derivative_change           46              50   

                                            Null Count  Min      25th  \
Feature Name      

### Q3 (25 points)
#### Drop features with improper cardinality:
Using the quality report summary table that is returned from the function you wrote for Q2, we are now going to investigate our data. For this, you should use the table returned for the [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv) and not the toy dataset I provided for testing.

Since we are using real valued features, a majority of them shall have a cardinality close to the sample count. So, for this question, you are to write a function that takes in the summary table and the input dataset DataFrame, and drops the feature that have a cardinality less than 10. This feature should be dropped from both the data quality report summary table and from the actual input dataset DataFrame.

A useful method for this operation is:

* [pandas.DataFrame.drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) (Make sure to use the inplace option otherwise it returns a copy)

In [ ]:
def drop_low_card_data(summary_table:DataFrame, data:DataFrame) -> None:
    
    i = summary_table.loc[summary_table['Cardinality'] < 10].index.tolist()
    summary_table.drop(i, inplace = True)
    data.drop(i, axis = 1, inplace = True)
    print(i)
    print(data)
    
    
    # Your answer to Q3 goes here!

In [9]:
drop_low_card_data(summary_table, abt)

['TOTUSJH_no_zero_crossings', 'TOTBSQ_no_zero_crossings', 'TOTPOT_no_zero_crossings', 'TOTUSJZ_no_zero_crossings', 'ABSNJZH_no_zero_crossings', 'SAVNCPP_no_zero_crossings', 'USFLUX_no_zero_crossings', 'TOTFZ_no_zero_crossings', 'MEANPOT_no_zero_crossings', 'EPSZ_no_zero_crossings', 'MEANSHR_no_zero_crossings', 'SHRGT45_no_zero_crossings', 'MEANGAM_no_zero_crossings', 'MEANGBT_no_zero_crossings', 'MEANGBZ_no_zero_crossings', 'MEANGBH_no_zero_crossings', 'TOTFY_no_zero_crossings', 'TOTFX_no_zero_crossings', 'EPSY_no_zero_crossings', 'EPSX_no_zero_crossings', 'R_VALUE_no_zero_crossings']


### Q4 (25 points)
#### Drop features with excessive NaN

Again, using the quality report summary table that is returned from the function you wrote for Q2, we are going to continue investigating our data. For this, you should still be using the table returned for the [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv) and not the toy dataset I provided for testing.

Like the features that were dropped for Q3, some of the extracted features don't work on all of the variates of the input multi-variate time series samples very well.  So, some of these features return an excessive number of not a number values.  These are not verry useful features, so we want to get rid of them before we continue. To do this, you are to write a function that takes in the summary table and the input dataset DataFrame, and drops the features that have more than 1% of the entries as null/nan values. Again, these features should be dropped from both the data quality report summary table and from the actual input dataset DataFrame.

As in Q3, a useful method for this operation is:

* [pandas.DataFrame.drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) (Make sure to use the inplace option otherwise it returns a copy)

In [82]:
def drop_excessive_nan_data(summary_table:DataFrame, data:DataFrame) -> None:
    empty=""
    the_list[]
    for i in summary_table['Null Count'].to_numpy():
        if ((i * 100) / len(summary_table['Null Count'].to_numpy()) > .01):
            the_list.append(empty)
        else:
            the_list.append(i)
    summary_table['Null Count'] = the_list
    locate = summary_table.loc[summary_table['Null Count'] == empty]
    summary_table.drop(locate, inplace=True)
    data.drop(locate, axis=1, inplace=True)
    print(data)
    # Your answer to Q4 here!

In [83]:
drop_excessive_nan_data(summary_table, abt)

    TOTUSJH_min  TOTUSJH_max  TOTUSJH_median  TOTUSJH_mean  TOTUSJH_stddev  \
0    103.728433   168.447571      125.901867    128.354127       15.077707   
1   3304.799233  4129.596653     3523.029578   3580.248772      190.250948   
2     13.292757    51.719175       29.843361     32.710697       12.338905   
3     94.138307   154.257141      111.495061    116.882144       18.198159   
4     24.938772    51.677508       40.623843     39.183960        6.368641   
5     18.528046    29.150427       23.787601     23.845579        2.364224   
6    203.451625   274.042767      226.589675    231.499210       16.510900   
7     25.429072    36.852150       30.096294     30.344992        2.449060   
8    279.397983   391.691371      353.734269    343.871652       35.560706   
9     51.705487    67.091729       59.893353     59.238280        4.370794   
10   114.733135   168.124143      138.336284    139.713102       11.686581   
11   118.578497   280.738636      214.478707    204.633954      

Now that you have the data cleaned up a little, save the results of both your summary table and your analytics base table using the [pandas.to_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) method. We will want to use these results for the next homework assignment.

In [89]:
df = pd.DataFrame()
out_dir = r'C:/Users/akash/OneDrive/Desktop/Data Science/hw2'
out_summary_table_name = r'data_quality_table.csv'
out_abt_name = r'cleaned_partition1ExtractedFeatures.csv'
df.to_csv(out_dir,out_summary_table_name)
df.to_csv(out_dir,out_abt_name)

In [86]:
# Save here!